# Using Metadata to Improve Artifical Intelligance Medical Image Diagnostic Accuracy
**Purpose and Background**
Conduct a descriptive analysis of crowdsourced data extracted from user interaction with a mobile application where tasked to binarly (yes or no) identify abnormalities in medical images. 

Two user categories were differentiated: Medical experts hired to interact with the application; and crowd, anyone who downloaded and used the application.

**Show that the crowd agrees with the expert majority more than experts agreeing with the expert majority**


### Import datasets

In [529]:
import pandas as pd
import numpy as np
results = pd.read_csv('1345_customer_results.csv') #medical case results
admin = pd.read_csv('1345_admin_reads.csv') #raw individual read

### Inspect Customer Results

In [530]:
results.dtypes
#results = results.set_index('Case ID')

Case ID                   int64
Origin                   object
Origin Created At        object
Content ID                int64
URL                      object
Labeling State           object
Series                  float64
Series Index            float64
Patch                   float64
Qualified Reads           int64
Correct Label            object
Majority Label           object
Difficulty              float64
Agreement               float64
First Choice Answer      object
First Choice Votes        int64
First Choice Weight     float64
Second Choice Answer     object
Second Choice Votes       int64
Second Choice Weight    float64
Internal Notes          float64
Comments                 object
Explanation             float64
dtype: object

**Preliminary filtering for security purposes**


In [531]:
results = results.dropna(subset=['Origin']) 
results["Expert: Abnormal Votes"] = results["Origin"].str.extract(r'vote(\d)').astype(float)
results = results.drop(['Origin Created At','Origin','Content ID','URL'],axis=1)

In [532]:
results.head(2)

,Case ID,Labeling State,Series,Series Index,Patch,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Internal Notes,Comments,Explanation,Expert: Abnormal Votes
0,5888087,Gold Standard,NaN,NaN,NaN,2,'no','no',0.0,1.0,'no',2,1.54,'yes',0,0.0,NaN,[],NaN,2.0
1,5888088,Gold Standard,NaN,NaN,NaN,3,'no','no',0.0,1.0,'no',3,2.34,'yes',0,0.0,NaN,[],NaN,0.0


Any rows that did not have a string associated with expert votes in the URL were dropped (i.e. NA)

*0 means all experts thought the case was normal  

In [533]:
results = results.dropna(subset=["Expert: Abnormal Votes"])
#results["Expert: Abnormal Votes"].isnull().any()

**Inspect NaN Columns for Content**

In [534]:
results.loc[results['Series'].notna()| results['Series Index'].notna() | results['Patch'].notna() | results['Internal Notes'].notna() | results['Explanation'].notna()]

,Case ID,Labeling State,Series,Series Index,Patch,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Internal Notes,Comments,Explanation,Expert: Abnormal Votes


Dataframe is empty; None of the columns scanned through the pipeline contained any data

In [535]:
results = results.drop(['Series','Series Index','Patch','Internal Notes','Explanation'],axis=1)

**Inspect Comments for Relevance**

In [536]:
results[results['Comments'] != '[]']


,Case ID,Labeling State,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Comments,Expert: Abnormal Votes
4245,5892332,Gold Standard,1,'no','no',0.000,1.000,'no',1,0.80,'yes',0,0.00,['There was rapid and spiky rates so why am I ...,3.0
6029,5894116,Gold Standard,5,'no','yes',1.000,1.000,'yes',5,4.00,'no',0,0.00,['Can someone explain why the answer is “no”?'],0.0
8346,5896433,Gold Standard,3,'yes','no',1.000,1.000,'no',3,2.32,'yes',0,0.00,['??'],5.0
11433,5899520,Gold Standard,2,'yes','no',1.000,1.000,'no',2,1.58,'yes',0,0.00,"[""i can't see any spike in this question so wh...",5.0
12911,5900998,Gold Standard,2,'no','yes',1.000,1.000,'yes',2,1.56,'no',0,0.00,['There is obviously a peak happened in there'],3.0
13827,5901914,Gold Standard,6,'yes','no',1.000,1.000,'no',6,4.72,'yes',0,0.00,['No spike present'],5.0
13953,5902040,Gold Standard,2,'no','yes',1.000,1.000,'yes',2,1.58,'no',0,0.00,['How?'],3.0
16033,5904120,Gold Standard,1,'yes','no',1.000,1.000,'no',1,0.78,'yes',0,0.00,['How? '],6.0
16326,5904413,Gold Standard,3,'yes','no',1.000,1.000,'no',3,2.46,'yes',0,0.00,['Multiple?'],6.0
16385,5904472,Gold Standard,3,'yes','yes',0.333,0.667,'yes',2,1.58,'no',1,0.78,[' Wtf'],5.0


None of the comments seem relevant, so the Comments column will be dropped

In [537]:
results = results.drop(['Comments'],axis=1)

There should only be 8 experts total; cases were dropped for experts count >8

In [538]:
results = results[results["Expert: Abnormal Votes"] <= 8]

### Important columns for analysis; original metadata
Each row corresponds to a medical case 

**Identifiers:** 

Case ID: unique identifier will serve as index

Labeling State: identifies whether a expert consensus has been achieved (yes=Gold Standard, no= In Progress)

URL: Extracted out expert vote count within the URL 

**Reads and Annotations**

Qualified Reads: total crowd vote count

Expert: Abnormal Votes: number of experts who thought the case was abnormal

(note, the total of experts voting is always 8)

Correct Label: overall expert consensus 

{yes=case is abnormal, no=case is normal, NaN=no consensus}

Majority Label: overall crowd consensus on each case

**Measures of Confidence**

Difficulty: Qualified Reads *without the Correct Label* divided by total Qualified Reads.

Agreement: Qualified Reads *with the Majority Label* divided by total Qualified Reads.

Nth Choice Answer: crowd answer (First Choice is the Majority Label)
        
Nth Choice Votes: number of crowd votes per answer
        
Nth Choice Weight:
        
        
        



### Add Additional Relevant Columns 




In [539]:
df = results 
expert_count = 8
df["Expert: Normal Votes"] = (expert_count - results["Expert: Abnormal Votes"])
df["Expert Agreement"] = df["Expert: Abnormal Votes"]/expert_count
#split_conses = (df.loc(df["Expert/Expert Agreement"]==0.5))
df

,Case ID,Labeling State,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Expert: Abnormal Votes,Expert: Normal Votes,Expert Agreement
0,5888087,Gold Standard,2,'no','no',0.000,1.000,'no',2,1.54,'yes',0,0.00,2.0,6.0,0.250
1,5888088,Gold Standard,3,'no','no',0.000,1.000,'no',3,2.34,'yes',0,0.00,0.0,8.0,0.000
2,5888089,Gold Standard,2,'no','no',0.000,1.000,'no',2,1.70,'yes',0,0.00,0.0,8.0,0.000
3,5888090,Gold Standard,1,'no','no',0.000,1.000,'no',1,0.82,'yes',0,0.00,0.0,8.0,0.000
4,5888091,In Progress,7,NaN,'yes',NaN,0.571,'yes',4,3.28,'no',3,2.32,4.0,4.0,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30288,5918375,Gold Standard,2,'no','yes',1.000,1.000,'yes',2,1.56,'no',0,0.00,2.0,6.0,0.250
30289,5918376,Gold Standard,3,'no','yes',0.667,0.667,'yes',2,1.56,'no',1,0.76,3.0,5.0,0.375
30290,5918377,In Progress,6,NaN,'yes',NaN,1.000,'yes',6,4.78,'no',0,0.00,4.0,4.0,0.500
30291,5918378,Gold Standard,0,'yes',NaN,NaN,NaN,'yes',0,0.00,'no',0,0.00,5.0,3.0,0.625


#### I will rename some of the original columns for clarity
JUST KIDDING :) (go back and change it, you can always make a mental note)

    {Original column --> Renamed Column}
    
    Correct Label --> Expert Majority

    Majority Label --> Crowd Majority

    Difficulty --> Expert/Crowd Disagreement

    Agreement --> Crowd Agreement

In [540]:
df["Expert Majority"] = results["Correct Label"]
df["Crowd Majority"] = results["Majority Label"]
df["Expert/Crowd Disagreement"] = results["Difficulty"] #porportion of crowd disagreeing with expert consensus
df["Crowd Agreement"] = results["Agreement"] #porportion of crowd agreeing with crowd consensus
df = df.drop(columns= ["Correct Label","Majority Label","Difficulty","Agreement"])


df['Consensus'] = np.where(df['Expert Majority'] == df['Crowd Majority'],'yes','no')

#### Error rate of experts
I extracted the indexes for each category and calculated the "error rate" for the experts who did not vote for the expert majority

In [576]:
EM_yes = df.index[df['Expert Majority'] == "'yes'"].tolist()

EM_no = df.index[df['Expert Majority'] == "'no'"].tolist()

df.loc[EM_yes,"Error Rate"]= df['Expert: Normal Votes'][EM_yes]/expert_count
df.loc[EM_no,"Error Rate"]= df['Expert: Abnormal Votes'][EM_no]/expert_count
df.fillna('', inplace=True)
beg_index = list(df.columns).index('Expert: Abnormal Votes') #9
df.iloc[ : , 13:]


,Crowd Majority,Expert/Crowd Disagreement,Crowd Agreement,Consensus,Error Rate
0,'no',0.0,1.0,yes,0.25
1,'no',0.0,1.0,yes,0.0
2,'no',0.0,1.0,yes,0.0
3,'no',0.0,1.0,yes,0.0
4,'yes',,0.571,no,
...,...,...,...,...,...
30288,'yes',1.0,1.0,no,0.25
30289,'yes',0.667,0.667,no,0.375
30290,'yes',,1.0,no,
30291,,,,no,0.375


## Exploratory Analysis 1

In [595]:
#%pip install jupyter-dash
import plotly.express as px
import plotly.io as pio
import plotly.figure_factory as ff
pio.renderers.default='notebook'
import matplotlib.pyplot as plt
#ax = plt.subplot()

In [543]:
#FIX NaN; currently is blank
df.fillna('', inplace=True)
print(df['Expert Majority'].value_counts())

'no'     12000
'yes'    12000
          3000
Name: Expert Majority, dtype: int64


12,000 medical cases were judged to be abnormal by experts

12,000 medical cases were judged to be normal by experts

3,0000 medical cases failed to reach a consensus (4 experts voted for normal and 4 experts voted for abnormal)


### Exploratory Analysis

In [692]:
fig = px.histogram(df,x= "Qualified Reads", color="Error Rate")
fig.update_layout(xaxis_range=[0,15])
fig.show()

In [695]:
print(len(df[df['Qualified Reads'] < 5].value_counts()))


12113


#### How reliable are the individual experts on average?

In [708]:

fig = px.density_heatmap(df, x="Expert Majority", y='Crowd Majority',text_auto=True)
fig.show()
df.std(axis=0)


/var/folders/qt/mmxjlbbn0lzdwmjghjtjwlc00000gn/T/ipykernel_24751/3685616616.py:3: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



Case ID                   7944.370156
Qualified Reads              2.549289
First Choice Votes           2.360711
First Choice Weight          1.888224
Second Choice Votes          0.959345
Second Choice Weight         0.764048
Expert: Abnormal Votes       2.582037
Expert: Normal Votes         2.582037
Expert Agreement             0.322755
dtype: float64

When the experts are undecided (N=4) on the case prognosis, crowd appears to have a more unified opinion on the case. Let's make a histogram examining the cases where there's lack of consensus.

With the number of experts being 8, filtering the qualified reads to 5 or more would keep things more porportional

In [709]:
filt_df = df[df['Qualified Reads'] >= 5]
fig = px.density_heatmap(filt_df, x="Expert Majority", y='Crowd Majority',text_auto=True)
fig.show()
filt_df.std(axis=0)

/var/folders/qt/mmxjlbbn0lzdwmjghjtjwlc00000gn/T/ipykernel_24751/701681104.py:4: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



Case ID                   7991.569566
Qualified Reads              1.942279
First Choice Votes           1.961178
First Choice Weight          1.569474
Second Choice Votes          1.137077
Second Choice Weight         0.905635
Expert: Abnormal Votes       2.456104
Expert: Normal Votes         2.456104
Expert Agreement             0.307013
Crowd Agreement              0.153559
dtype: float64

In [696]:
fig = px.scatter(filt_df,x= "Expert Agreement", y="Crowd Agreement")
#fig.update_layout(yaxis_range=[0.4,1.1])
fig.show()

This doesn't tell us much; perhaps calculate an average?

In [701]:
consensus_no = filt_df.index[filt_df['Consensus']=='no'].tolist()
fig = px.histogram(filt_df.loc[consensus_no], x='Crowd Agreement',color='Crowd Majority',marginal='box')
fig.update_layout(xaxis_range=[-0.1,1.1])
fig.show()

Move red to left side of the graph

In [704]:
fig = px.histogram(filt_df.loc[consensus_no], x='Expert Agreement',color='Expert Majority',marginal='box')
fig.show()

### Let's check out what are the trends when experts are split in their opinion:
what about when crowd is split?

In [706]:
experts_split = filt_df.index[filt_df['Expert Agreement']==0.500].tolist()
fig = px.histogram(filt_df.loc[experts_split], x='Crowd Agreement',color='Crowd Majority',marginal='box')
fig.update_layout(xaxis_range=[0.45,1.05])
fig.show()

## Trying to aggregate case ids

In [553]:

filt_df = df[df['Qualified Reads'] > 5]
fig = px.histogram(filt_df,x= "Case ID")
#fig.update_layout(yaxis_range=[0.4,1.1])
fig.show()

per case ID:
sum the instances and show porportions of error rate; ### aggregate case IDs!!

In [554]:
import statistics
#filt_df['aggreg_error_rate'] = filt_df.groupby(['Case ID'], as_index = False)['Error Rate'].mean()
#filt_df['aggreg_error_rate'] = filt_df.groupby(['Case ID'], as_index = False).apply(lambda x: statistics.mean(x))
def average(x):
    x/sum(x)
#print(average(df['Expert: Normal Votes']))
#caseid_agg = df.groupby(['Case ID'])['Expert Agreement'].aggregate{}
print(df['Case ID'].unique)
#fig = px.histogram(caseid_agg)
#fig.show()
#print(sum(caseid_agg))
print(df.index[-1] - df.index[1])


<bound method Series.unique of 0        5888087
1        5888088
2        5888089
3        5888090
4        5888091
          ...   
30288    5918375
30289    5918376
30290    5918377
30291    5918378
30292    5918379
Name: Case ID, Length: 27000, dtype: int64>
30291


In [710]:

filt_df = df[df['Qualified Reads'] > 5]
fig = px.histogram(filt_df,x="Case ID", color="Error Rate")
#fig.update_layout(yaxis_range=[0.45,1.1])
fig.show()

Seems uniform; which cases fall into the category? Are they the harder ones?

Make these into bubble charts to show porportion: https://plotly.com/python/bubble-charts/